# Bulk Delete of Members in TeamApp

This robot notebook deletes a set of members from the club's TeamApp. This is used to delete all members that are not part of the new season.

[![https://www.youtube.com/watch?v=0pYoiOhjskY](https://img.youtube.com/vi/0pYoiOhjskY/0.jpg)](https://www.youtube.com/watch?v=0pYoiOhjskY)

In [1]:
import re

# Download geckodriver (https://github.com/mozilla/geckodriver/releases) and put it in path
# Salenium webdriver: https://www.selenium.dev/documentation/overview/
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By

## Create Virtual Browser

Create a firefox instance using an existing profile that MUST be logged into TeamApp.

Set the following variables and in particular `FIREFOX_PROFILE_PATH` to the existing Firefox profile that is already logged into the club's TeamApp.

In [2]:
# URL of the club's TeamApp
URL_TEAMAPP = "https://brunswickmagicbasketball.teamapp.com"

# Folder with the firefox profile to be used
FIREFOX_PROFILE_PATH = '/home/ssardina/.mozilla/firefox/88weweww.ssardina'

GECKODRIVER_PATH = '/home/ssardina/bin/geckodriver'

In [ ]:
URL_CHATS = f"{URL_TEAMAPP}/rooms?_list=v1"

# firefox profile of user already logged in teamapp

options = Options()
# options.headless = True
options.page_load_strategy = 'eager'
options.set_preference('profile', FIREFOX_PROFILE_PATH)

# https://github.com/SeleniumHQ/selenium/issues/11028
# use an existing firefox profile (must be logged in teamapps already!)
options.add_argument("-profile")
options.add_argument(FIREFOX_PROFILE_PATH)

service = None
# service = Service(GECKODRIVER_PATH)

if service is not None:
    browser = webdriver.Firefox(service=service, options=options)
else:
    browser = webdriver.Firefox(options=options)
# browser.maximize_window()

browser.get(URL_CHATS)

# browser.quit()


In case you want to close it:

In [ ]:
browser.quit()

# Test delete on member

We start by retrieving the member card:

In [ ]:
member = 12240523

browser.get(f"{URL_MEMBERS}/{member}")

if browser.title == "Page Not Found":
    print("Member not found!")


Get the **Advance Option** three dot button at the top-right. The `XPATH` was found by inspecting the DOM. 

This will give an object of [selenium.webdriver.remote.webelement.WebElement](https://www.selenium.dev/selenium/docs/api/py/webdriver_remote/selenium.webdriver.remote.webelement.html).

For this button, we search via its fixed `XPATH`: https://selenium-python.readthedocs.io/locating-elements.html


## 1. Click Advanced Options button

![](members_01.png)

In [ ]:
# click the Advanced Options button (three dots top right of member)
button_options_xpath = "/html/body/div[1]/div[1]/main/div/div[2]/div/header/div/span[2]/button"

button_options = browser.find_elements(by=By.XPATH, value=button_options_xpath)[0]
print(button_options)

Now click it to open options

In [ ]:
button_options.click()

## 2. Click Delete button

![](members_02.png)

Next, find the `Delete` button and click it!

In [ ]:
button_delete = None
for x in browser.find_elements(By.XPATH, "//*[contains(text(),'Delete')]"):
    if x.text == "Delete":
        button_delete = x
        break

if button_delete is not None:
    print("Delete button found!")

In [ ]:
button_delete.click()

## 3. Confirm Delete

![confirm delete](members_03.png)

In [ ]:
button_confirm = None
for x in browser.find_elements(By.XPATH, "//*[contains(text(),'Delete')]"):
    print(x, x.text, x.get_attribute("class"), x.get_dom_attribute("class"))
    if x.text == "Delete" and x.get_attribute("class") == "v-btn__content":
        button_confirm = x
        break

if button_confirm is not None:
    print("Delete button found!")

In [ ]:
button_confirm.click()

# Bulk delete members


In [ ]:
import time

members_all = [19789861,19789799,19789802,16550015,19789815,18623151,18703651,19789813,19867371,20967657,15002114,20251630,14631329,15169318,18764760,19789826,18623102,14605074,15169298,18623101,12456533,19789836,19789845,18758834,19789800,19789848,16296663,19252320,16296667,19789858,12240512,19789882,8995680,19789866,19789849,18623123,12240511,19789871,19789894,18703652,19789859,15215223,19789801,18623120,19928890,19928889,18623122,5767257,19841627,16309428,20251631,12240523,19789820,18623103,19928891,19841630,16296664,19789880,19811367,19841631,16352517,19789837,19789850,16352518,12240524,19789838,19789804,19811150,19789892,18623150,16296665,14605072,19789872,19789846,15214993,19811149,19789881,18623121,19789883,19811368,19789814,19789950,19789847,16296668,19867366,16352523,15002120,19789949,19789951,16352522,19811366,19789952,19789807,19789827,13319754,19789893,16352521,18623149,18623152]

print(len(members_all))

In [ ]:
members = members_all[28:]

print(f"About to delete {len(members)} members:", members)

deleted = []
for m in members:
    url_member = f"{URL_MEMBERS}/{m}"
    print("Processing member:", m,  url_member)
    browser.get(url_member)

    if browser.title == "Page Not Found":
        print("\t Member not found!")
        continue

    # click the Advanced Options button (three dots top right of member)
    print("\t Clicking advanced option button...")
    button_options_xpath = "/html/body/div[1]/div[1]/main/div/div[2]/div/header/div/span[2]/button"

    button_options = browser.find_elements(by=By.XPATH, value=button_options_xpath)[0]
    button_options.click()


    print("\t Find and click Delete button...")
    button_delete = None
    for x in browser.find_elements(By.XPATH, "//*[contains(text(),'Delete')]"):
        if x.text == "Delete":
            button_delete = x
            break
    if button_delete is None:
        print("\t Delete button NOT found!")
        continue
    button_delete.click()

    print("\t Confirm delete...")
    button_confirm = None
    for x in browser.find_elements(By.XPATH, "//*[contains(text(),'Delete')]"):
        if x.text == "Delete" and x.get_attribute("class") == "v-btn__content":
            button_confirm = x
            break

    if button_confirm is None:
        print("\t Delete confirm button NOT found!")
        continue

    button_confirm.click()
    deleted.append(m)
    time.sleep(2)




not_deleted = [x for x in members_all if x not in deleted]
print(f"Deleted members ({len(deleted)}):", deleted)
print(f"Not deleted members ({len(not_deleted)}):", not_deleted)


In [ ]:
browser.quit()

## Playground

In [ ]:
for x in browser.find_elements(By.XPATH, "//*[contains(text(),'Delete')]"):
        if x.text == "Delete" and x.get_attribute("class") == "v-btn__content":
            button_confirm = x
            break